# functions

In [4]:
import os
os.environ["OMP_NUM_THREADS"] = str(1)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit, minimize
from multiprocessing import Pool, cpu_count
import tqdm
import random
from pathlib import Path
import torch
import robust_loss_pytorch
import pandas as pd

In [5]:
def caf(tt,k1,k2,k3,k4,xxx0):
    y = ((-np.exp(-(k3+k2)*tt)+np.exp(-k1*tt))*k1)/(k3+k2-k1)
    return y

def xxx(tt,k1,k2,k3,k4,xxx0):
    y = (np.exp(-(k1+k4)*tt)*(-np.exp(k4*tt)*k2*k1*(k3+k2-k4)+np.exp((-k3-k2+k1+k4)*tt)*k2*k1*(k1-k4)-np.exp(k1*tt)*(-k3-k2+k1)*((k3-k4)*(k1-k4)*xxx0+k2*(k1+k1*xxx0-k4*xxx0))))/((k3+k2-k1)*(k3+k2-k4)*(k1-k4))
    return y

In [6]:
def load_imputed_data(donors):
    '''
    Loads (normalized by machine standard) raw data from a list of donors.
    Returns DataFrames in a dictionary.
    '''
    
    df = pd.read_csv('/home/users/mgotsmy/sweat/210000_notebooks/210706_A_train_test_set/train_data/4_with_metadata.csv',index_col=0)
    data = {}
    for donor in donors:
        tmp = df[df['donor']==donor]
        data[donor] = tmp.loc[:,['time','Caffeine','Paraxanthine','Theobromine','Theophylline']]
    
    return data

def optimization_problem_robust_loss_production(_input):
    '''
    Uses _input list of donor_name, sampled p0, and bounds (lb, ub). Calls curve_fit() and writes out the fitted parameters.
    '''
    
    # the loss parameters alpha and scale are initially estimated as 1
    global alpha, scale, loss
    alpha, scale = 1, 1
    # reading the input
    donor, p0, lb, ub, metabolite = _input
    # doing data preprocessing
    x_values, y_values, s_values = preprocessing_imputed_data(data,donor,metabolite)

    # in rare cases initial parameters can lead to NaNs which can create errors. This is excepted here.
    try:
        para, var = curve_fit(f         = fit_with_sv,
                              xdata     = x_values,
                              ydata     = y_values,
                              sigma     = s_values,
                              p0        = p0,
                              bounds    = (lb.astype('float64') ,ub.astype('float64')),
                              method    = 'trf',
                              max_nfev  = 1000,
                              loss      = max_cauchy_loss,
                              tr_solver = 'exact')
        observed = y_values
        expected = fit_with_sv(x_values,*para)
        loss = np.sum(max_cauchy_loss((np.abs(observed-expected)/s_values)**2)[0])
        return [donor,metabolite, list(para), float(loss)]
    except (ValueError,RuntimeError) as e:
        return '#Error'
    
def optimization_problem_robust_loss_debug(_input):
    '''
    Uses _input list of donor_name, sampled p0, and bounds (lb, ub). Calls curve_fit() and writes out the fitted parameters.
    '''
    
    # the loss parameters alpha and scale are initially estimated as 1
    global alpha, scale, loss
    alpha, scale = 1, 1
    # reading the input
    donor, p0, lb, ub, metabolite = _input
    # doing data preprocessing
    x_values, y_values, s_values = preprocessing_imputed_data(data,donor,metabolite)

    # in rare cases initial parameters can lead to NaNs which can create errors. This is excepted here.
#     try:
    para, var = curve_fit(f         = fit_with_sv,
                          xdata     = x_values,
                          ydata     = y_values,
                          sigma     = s_values,
                          p0        = p0,
                          bounds    = (lb.astype('float64') ,ub.astype('float64')),
                          method    = 'trf',
                          max_nfev  = 1000,
                          loss      = max_cauchy_loss,
                          tr_solver = 'exact')
    observed = y_values
    expected = fit_with_sv(x_values,*para)
    loss = np.sum(max_cauchy_loss((np.abs(observed-expected)/s_values)**2)[0])
    return [donor,metabolite, list(para), float(loss)]
#     except (ValueError,RuntimeError) as e:
#         return '#Error'

def preprocessing_imputed_data(data,donor,metabolite):
    '''
    Takes raw data dictionary and donor name.
    Parses x_values and y_values.
    Transforms y_values by calibration curve.
    Assigns 10**8 sigma values (= weights) to all 0s because they are below LOQ and thus regarded as NaNs.
    Returns x_values, sigma, y_values as float64 np.arrays.
    '''
    
    x_values = data[donor]['time'].copy()
    y_values = np.array(data[donor].loc[:,['Caffeine','Paraxanthine','Theobromine','Theophylline']].copy())
    y_values = y_values.flatten()
    
    # apply calibration curve
    calibration_k = np.array([np.ones(len(x_values))*1.520,np.ones(len(x_values))*1.656,np.ones(len(x_values))*2.050,np.ones(len(x_values))*1.592]).flatten('F')
    y_values      = y_values*calibration_k

    # pharmacological factors
    vdist_avail = 0.579                                        # volume of distribution/ bioavailability of caffeine
    vsample     = 123                                          # sample volume in µL
    dose        = 200*10**3                                    # caffeine dose in µg
    body_mass   = {'Donor_16': 72, 'Donor_6': 83, 'Donor_19': 83, 'Donor_17': 52, 'Donor_18': 66, 'Donor_7': 66, 'Donor_3': 57, 'Donor_4': 84, 'Donor_5': 82.5, 'Donor_2': 77, 'Donor_8': 80, 'Donor_9': 83, 'Donor_10': 55, 'Donor_11': 54, 'Donor_1': 70, 'Donor_20': 105, 'Donor_21': 70, 'Donor_22': 64, 'Donor_23': 57, 'Donor_24': 60, 'Donor_25': 80, 'Donor_26': 86, 'Donor_27': 68, 'Donor_28': 82, 'Donor_29': 60, 'Donor_30': 55, 'Donor_31': 80, 'Donor_32': 92, 'Donor_33': 92, 'Donor_34': 71, 'Donor_35': 80, 'Donor_36': 77, 'Donor_37': 66, 'Donor_38': 63, 'Donor_39': 75, 'Donor_40': 75, 'Donor_41': 57, 'Donor_42': 75, 'Donor_43': 99, 'Donor_44': 75, 'Donor_45': 61, 'Donor_46': 52, 'Donor_47': 80}
    factor      = vsample*body_mass[donor]*vdist_avail/dose
    y_values    = y_values*factor
    
    

    if metabolite == 'Paraxanthine':
        i = 1
    elif metabolite == 'Theobromine':
        i = 2
    elif metabolite == 'Theophylline':
        i = 3
    else:
        print('unkown metabolite:',metabolite)
    
    # applying lambda 
    sigma   = []
    if lambda_method == 'n_PQN':
        n_lambda = len(pqn_dic[donor][metabolite])
    elif lambda_method == 'n_EM':
        n_lambda = 2
    else:
        print('Unknown lambda_method:',lambda_method)
    lambda_ = 1/(1+n_lambda)
    concentration_sigma = np.ones(len(y_values))/lambda_
    pqn_sigma           = np.ones(len(pqn_dic[donor][metabolite]))/(1-lambda_)
    y_values = np.concatenate([y_values.reshape(-1,4)[:,[0,i]].reshape(-1),pqn_dic[donor][metabolite]])
    s_values = np.concatenate([concentration_sigma.reshape(-1,4)[:,[0,i]].reshape(-1),pqn_sigma]) 
    
    
    return x_values.astype('float64'), y_values.astype('float64'), s_values.astype('float64')

def save_data(output_list,filepath):
    '''
    Takes list of fitting results and file path.
    Saves fitting results row-wise in the given path.
    '''
    
    with open(filepath,'w') as file:
        for ol in output_list:
            file.write(str(ol).replace('[','').replace(']','').replace("'",'').replace(',','')+'\n')
    return

def fit_with_sv(tt,x,k1,k2,k3,k4,xxx0,*sfs):
    '''
    This function takes timepoints (tt) and all fitting parameters and calculates original values (c*V_sweat) with it. 
    '''
    
    # global_args and global_timepoints are needed for the calculation of relative fitting error
    global global_args, global_timepoints, global_sfs, global_x
    args = [k1,k2,k3,k4,xxx0]
    global_args = args
    global_timepoints = tt
    global_sfs = sfs
    global_x = x
    
    # calculated c*V_sweat
    caf_true = np.array(caf(tt,*args))*np.array(sfs)
    xxx_true = np.array(xxx(tt,*args))*np.array(sfs)
    y = np.array([caf_true,xxx_true])
    y = np.concatenate([y.flatten('F'),np.array(sfs)*x])
    return y    

def fit_without_sv(tt,x,k1,k2,k3,k4,xxx0,*sfs):
    '''
    This function takes timepoints and all fitting parameters and calculates concentrations (c) with it. 
    Only for single measurements per timepoint!
    '''
    args = [k1,k2,k3,k4,xxx0]
    caf_true = np.array(caf(tt,*args))
    xxx_true = np.array(xxx(tt,*args))
    y = np.array([caf_true,xxx_true])
    y = np.concatenate([y.flatten('F'),np.array(sfs)*x])
    return y

def max_cauchy_loss(z):
    '''Takes array, calculates maximum of relative and absolute error and
    calculates Cauchy loss [2].
    '''

    global alpha, scale, loss
    abs_error = z
    # calculate relative error, except absolute error is 0
    y = fit_without_sv(global_timepoints,global_x,*global_args,*global_sfs)
    rel_error = np.divide(z, y, out=z.copy(), where=y!=0)
    # maximum error
    z = np.maximum(abs_error,rel_error)

    rho = np.empty((3,len(z)))
    rho[0] = np.log1p(z)
    t = 1 + z
    rho[1] = 1 / t
    rho[2] = -1 / t**2
    return rho

# calculate pqn

In [7]:
def calculate_pqn(data):
    # implemented according to http://dx.doi.org/10.1016/j.chroma.2014.08.050
    ref = np.median(data,axis=1) 
    div = np.divide(data,ref[:,None])
    pqn = np.median(div,axis=0)
    return pqn

# set rng seed
np.random.seed(13)

# load untargeted data
df = pd.read_csv('/home/users/mgotsmy/sweat/210000_notebooks/210706_A_train_test_set/train_data/4_with_metadata.csv',index_col=0)
donors = ['Donor_1', 'Donor_10', 'Donor_16', 'Donor_17', 'Donor_18', 'Donor_19', 'Donor_2', 'Donor_20', 'Donor_21', 'Donor_22', 'Donor_23', 'Donor_24', 'Donor_25', 'Donor_26', 'Donor_27', 'Donor_28', 'Donor_29', 'Donor_3', 'Donor_30', 'Donor_31', 'Donor_32', 'Donor_33', 'Donor_34', 'Donor_35', 'Donor_36', 'Donor_37', 'Donor_38', 'Donor_39', 'Donor_4', 'Donor_40', 'Donor_41', 'Donor_42', 'Donor_43', 'Donor_44', 'Donor_45', 'Donor_46', 'Donor_47', 'Donor_5', 'Donor_7', 'Donor_8', 'Donor_9']

# create PQN normalizing factors for 3 caffeine degradation metabolites
m_list = np.arange(58)
pqn_dic = {}
for donor in donors:
    tmp = df[df['donor']==donor]
    tmp = tmp.iloc[:,9:].values.T
    np.random.shuffle(m_list)
    split_lists = np.array_split(m_list,3)
    pqn_dic[donor] = {}
    for nr, metabolite in enumerate(['Paraxanthine','Theobromine','Theophylline']):
        pqn = calculate_pqn(tmp[split_lists[nr],:])
        pqn_dic[donor][metabolite] = pqn

In [9]:
## sampling MC replicates

# load raw data for all donors
donors = ['Donor_1', 'Donor_10', 'Donor_16', 'Donor_17', 'Donor_18', 'Donor_19', 'Donor_2', 'Donor_20', 'Donor_21', 'Donor_22', 'Donor_23', 'Donor_24', 'Donor_25', 'Donor_26', 'Donor_27', 'Donor_28', 'Donor_29', 'Donor_3', 'Donor_30', 'Donor_31', 'Donor_32', 'Donor_33', 'Donor_34', 'Donor_35', 'Donor_36', 'Donor_37', 'Donor_38', 'Donor_39', 'Donor_4', 'Donor_40', 'Donor_41', 'Donor_42', 'Donor_43', 'Donor_44', 'Donor_45', 'Donor_46', 'Donor_47', 'Donor_5', 'Donor_7', 'Donor_8', 'Donor_9']
data   = load_imputed_data(donors)
# create output directory
lambda_method = 'n_PQN' # n_EM, n_PQN
path = f'MIX_sub_2_{lambda_method}'
Path(path).mkdir(exist_ok=True)

# generate MC replicates
input_list  = []
output_list = []

for metabolite in ['Paraxanthine','Theobromine','Theophylline']:
    for donor in donors:
        # set random seed to donor number
        random.seed(float(donor.split('_')[1]))
        # number of sweat volumes that are needed for fitting
        n_sf = len(data[donor].index)
        # calculate the normalized c(0) for C(0) = 1 mg/L for all donors
        dose        = 200*10**3    # caffeine dose
        body_mass   = {'Donor_16': 72, 'Donor_6': 83, 'Donor_19': 83, 'Donor_17': 52, 'Donor_18': 66, 'Donor_7': 66, 'Donor_3': 57, 'Donor_4': 84, 'Donor_5': 82.5, 'Donor_2': 77, 'Donor_8': 80, 'Donor_9': 83, 'Donor_10': 55, 'Donor_11': 54, 'Donor_1': 70, 'Donor_20': 105, 'Donor_21': 70, 'Donor_22': 64, 'Donor_23': 57, 'Donor_24': 60, 'Donor_25': 80, 'Donor_26': 86, 'Donor_27': 68, 'Donor_28': 82, 'Donor_29': 60, 'Donor_30': 55, 'Donor_31': 80, 'Donor_32': 92, 'Donor_33': 92, 'Donor_34': 71, 'Donor_35': 80, 'Donor_36': 77, 'Donor_37': 66, 'Donor_38': 63, 'Donor_39': 75, 'Donor_40': 75, 'Donor_41': 57, 'Donor_42': 75, 'Donor_43': 99, 'Donor_44': 75, 'Donor_45': 61, 'Donor_46': 52, 'Donor_47': 80}
        vdist_avail = 0.579          # specific volume of distribution for caffeine
        factor      = (body_mass[donor]*vdist_avail*1000)/dose
        # set system bounds
        lb = np.concatenate(([0],np.zeros(5),np.ones(n_sf)*.05))                        # lower bounds
        ub = np.concatenate(([10,10],np.ones(3)*.2,np.ones(1)*factor,np.ones(n_sf)*4))  # upper bounds

        # sample MC replicates
        n_try   = 0
        n_tries = 100  # number of MC replicates used
        while n_try < n_tries:
            n_try += 1
            p0     = [] # initial parameters for the fit
            for l,u in zip(lb,ub):
                p0.append(random.uniform(l,u)) # sample from an uniform distribution between the system bounds
            input_list.append([donor,p0,lb,ub,metabolite])
            
# multiprocess fitting
with Pool(processes = cpu_count()) as p: # by default all processors are used
    err_count = 0
    for _ in tqdm.tqdm(p.imap_unordered(optimization_problem_robust_loss_production,input_list),total=len(input_list)):
        if _ == '#Error':
            err_count += 1
            pass
        else:
            output_list.append(_)
print('Error Percentage {:2.2f}'.format(err_count/len(input_list)*100))

# safe the results
for metabolite in ['Paraxanthine','Theobromine','Theophylline']:
    for donor in donors:
        write_out = []
        for i in output_list:
            if i[0] == donor and i[1] == metabolite:
                write_out.append(i[2:])
        if len(write_out) > 0:
            save_data(write_out,f'{path}/{donor}_{metabolite}.txt')

100%|██████████| 12300/12300 [07:56<00:00, 25.81it/s]


Error Percentage 0.04
